<a href="https://colab.research.google.com/github/research-clone/notebook_tutorials/blob/main/Paraphrasing_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install SentencePiece
!pip install accelerate -U
!pip install evaluate
!pip install rouge_score

NotImplementedError: ignored

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("humarin/chatgpt-paraphrases", split="train").shuffle(seed=42).select(range(50000))
test_dataset = load_dataset("humarin/chatgpt-paraphrases", split="train").shuffle(seed=42).select(range(50000, 55000))

In [ ]:
import torch, nltk
import evaluate
import numpy as np
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, DataCollatorWithPadding, Trainer, AutoModelForSeq2SeqLM, AutoTokenizer
nltk.download("punkt", quiet=True)

True

In [ ]:
model_name = 't5-small'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(torch_device)

In [ ]:
import ast
def preprocess_function(examples):
    inputs = [f"Paraphrase this sentence: {doc}" for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=100, truncation=True)
    labels = [ast.literal_eval(i)[0] for i in examples['paraphrases']]
    labels = tokenizer(labels, max_length=100, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, label_pad_token_id=-100, padding='longest')

In [ ]:
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=32,
    weight_decay=1e-3,
    save_total_limit=3,
    num_train_epochs=50,
    predict_with_generate=True
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,1.477600,1.272234,0.532236,0.289560,0.466126,0.467387
2,1.429700,1.252331,0.536542,0.293690,0.471028,0.472376


TrainOutput(global_step=3126, training_loss=1.5014735274946391, metrics={'train_runtime': 1028.8857, 'train_samples_per_second': 97.193, 'train_steps_per_second': 3.038, 'total_flos': 1831957018705920.0, 'train_loss': 1.5014735274946391, 'epoch': 2.0})

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
model.save_pretrained("")